# Decision Generator (Recency-Aware, Risk-Capped, Anchored)

This notebook generates next-round decision files:
- `data/prices/prices_<vNext>.json`
- `data/amounts/amounts_<vNext>.json`
- `data/schedules/schedules_<vNext>.json`

It reads `model_data.json` (created by `model_and_analysis_generator.ipynb`) and applies:
- **Anchor pricing from best-selling week** (per product)
- **Recency decay** (penalize items with long time since last sale)
- **Dead-item suppression** (stronger reductions for zero recent sales)
- **Trend-aware, risk-capped amounts**
- **Schedule picked by ROI** (revenue / staff cost)

Includes an **audit CSV** and quick anomaly checks (e.g., “dinosaur”).

In [19]:
# === Setup & Parameters ======================================================
import json, math
from pathlib import Path
import numpy as np
import pandas as pd

# Paths
DATA_DIR = Path("data")
SCHEDULES_DIR = DATA_DIR / "schedules"
OUT_DIR = Path("output"); OUT_DIR.mkdir(exist_ok=True, parents=True)

MODEL_DATA_PATH = Path("model_data.json")  # produced by model_and_analysis_generator

# Core parameters (conservative for competitive play)
PRICE_UP_SOLDOUT = 0.03     # +3% if very high sell-through
PRICE_UP_GOOD    = 0.02     # +2% for strong demand
PRICE_DOWN_BAD   = 0.05     # -5% for weak demand with stock
PRICE_DOWN_ZERO  = 0.10     # -10% for dead items

MIN_MARGIN_RATE  = 0.10     # ensure price >= (1 + MIN_MARGIN_RATE) * supplier_price
ROUND_CAP_STD    = 0.05     # ±5% per round cap
ROUND_CAP_DEAD   = 0.10     # ±10% per round cap for dead items

AMT_BASE_BUFFER  = 1.20     # base buffer over max observed weekly demand
AMT_BOOST_HOT    = 1.10     # extra +10% for super hot items
AMT_DEAD_FACTOR  = 0.50     # dead items keep 50% of average stock
RISK_CAP_MIN     = 1.10     # min cap multiplier on avg amount
RISK_CAP_MAX     = 1.50     # max cap multiplier on avg amount

EMA_ALPHA        = 0.6      # recency emphasis for sell-through EMA
RECENCY_DECAY    = 0.10     # 10% penalty per week since last sale (clamped)
RECENCY_MIN      = 0.60     # floor for recency multiplier (60% of base)

# Optional: product name filters for quick audits
ANOMALY_FILTERS = ["dinosaur", "matrat", "mattress"]

In [20]:
# === Load model data + helpers ==============================================
assert MODEL_DATA_PATH.exists(), "model_data.json not found. Run model_and_analysis_generator first."
md = pd.DataFrame(json.load(open(MODEL_DATA_PATH, "r", encoding="utf-8")))

VERSIONS = sorted(md["version"].unique())
V_NOW = VERSIONS[-1]
HIST_VERS = [v for v in VERSIONS if v < V_NOW]
V_NEXT = V_NOW + 1

print("Versions:", VERSIONS, "| Learn from:", HIST_VERS, "| Next:", V_NEXT)

def safe_div(a, b):
    a = np.asarray(a, float); b = np.asarray(b, float)
    return np.divide(a, b, out=np.zeros_like(a), where=b!=0)

def ema_series(x, alpha=EMA_ALPHA):
    s = None
    for xi in x:
        s = alpha*xi + (1-alpha)*(s if s is not None else xi)
    return s if s is not None else 0.0

Versions: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)] | Learn from: [np.int64(0), np.int64(1), np.int64(2), np.int64(3)] | Next: 5


In [21]:
# === Build historical table & select anchors =================================
g = (md.groupby(["version","product"], as_index=False)
        .agg(qty=("qty","sum"),
             amount=("amount","mean"),
             price=("price","mean"),
             revenue=("revenue","sum"),
             supplier_price=("supplier_price","mean")))

g_hist = g[g["version"].isin(HIST_VERS)].copy()

# Anchor: best-selling week by qty, then higher revenue, then lower price
g_sorted = g_hist.sort_values(["product","qty","revenue","price"],
                              ascending=[True, False, False, True])
anchor = (g_sorted.groupby("product", as_index=False).first()
                 [["product","version","price","qty","revenue","amount","supplier_price"]]
                 .rename(columns={"version":"anchor_v","price":"p_anchor",
                                  "qty":"anchor_qty","revenue":"anchor_revenue",
                                  "amount":"anchor_amount",
                                  "supplier_price":"supplier_price_ref"}))

print("Anchor sample:")
display(anchor.head(8))

Anchor sample:


,product,anchor_v,p_anchor,anchor_qty,anchor_revenue,anchor_amount,supplier_price_ref
0,batteries,2,15.85,2737.0,43381.45,2737.0,12.0
1,dinosaur,1,94.29,448.0,42241.92,1678.0,65.0
2,gjokur_ja,3,867.60,69.0,59864.40,69.0,580.0
3,hammer,1,66.54,3339.0,222177.06,3339.0,45.0
4,hot_dogs,0,26.25,2240.0,58800.00,2240.0,18.0
5,ice_cream,1,10.68,2864.0,30587.52,2864.0,8.0
6,knives,2,52.51,1356.0,71203.56,3236.0,38.0
7,laderhosen,1,316.95,345.0,109347.75,1686.0,220.0


In [22]:
# === Indicators: EMA sell-through, VOL, Risk, Recency ========================
# Per-product historical indicators
hist = (g_hist.groupby("product")
          .apply(lambda d: pd.Series({
              "ST_EMA": ema_series(safe_div(d["qty"], np.maximum(d["amount"],1))),
              "qty_max": d["qty"].max(),
              "amount_avg": d["amount"].mean(),
              "VOL": float(d["qty"].std(ddof=0)) if len(d) > 1 else 0.0
          }))
          .reset_index())

# Medians for risk scaling
p_med = g_hist.groupby("product")["price"].median().rename("price_med")
c_med = g_hist.groupby("product")["supplier_price"].median().rename("cost_med")

ref = (anchor.merge(hist, on="product", how="left")
             .merge(p_med, on="product", how="left")
             .merge(c_med, on="product", how="left"))

ref["PCR"] = safe_div(ref["p_anchor"], ref["supplier_price_ref"])
ref["PCR_med"] = safe_div(ref["price_med"], ref["cost_med"])

# Risk score: higher → more cautious amounts
w1, w2, w3 = 0.5, 0.3, 0.2
VOL_med = ref["VOL"].median() if ref["VOL"].median() > 0 else 1.0
ref["RISK"] = (
    (ref["PCR"] / ref["PCR_med"]).replace([np.inf, -np.inf], np.nan).fillna(1.0)*w1 +
    (ref["p_anchor"] / ref["price_med"]).replace([np.inf, -np.inf], np.nan).fillna(1.0)*w2 +
    (ref["VOL"] / VOL_med).replace([np.inf, -np.inf], np.nan).fillna(1.0)*w3
)

# Dead items: no sales in last 2 historical weeks (if exist)
last_weeks = sorted(HIST_VERS)[-2:] if len(HIST_VERS) >= 2 else HIST_VERS
recent = g_hist[g_hist["version"].isin(last_weeks)].groupby("product")["qty"].sum()
ref = ref.merge(recent.rename("qty_recent"), on="product", how="left").fillna({"qty_recent":0.0})
ref["is_dead"] = ref["qty_recent"] <= 0

# Recency: weeks since last non-zero sale
last_sale_v = (g[g["qty"]>0].groupby("product")["version"].max().rename("last_sold_version"))
ref = ref.merge(last_sale_v, on="product", how="left")
ref["weeks_since_sale"] = V_NOW - ref["last_sold_version"].fillna(0)
# Recency decay multiplier: 1 - 0.1 * weeks_since_sale, clamped to [RECENCY_MIN, 1.0]
ref["recency_multiplier"] = (1.0 - RECENCY_DECAY*ref["weeks_since_sale"]).clip(lower=RECENCY_MIN, upper=1.0)

print("Indicators sample:")
display(ref.head(8))

Indicators sample:


/tmp/ipykernel_22087/433232762.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({


,product,anchor_v,p_anchor,anchor_qty,anchor_revenue,anchor_amount,supplier_price_ref,ST_EMA,qty_max,amount_avg,...,price_med,cost_med,PCR,PCR_med,RISK,qty_recent,is_dead,last_sold_version,weeks_since_sale,recency_multiplier
0,batteries,2,15.85,2737.0,43381.45,2737.0,12.0,1.000000,2737.0,2131.25,...,16.090,12.0,1.320833,1.340833,0.982917,4562.0,False,4,0,1.0
1,dinosaur,1,94.29,448.0,42241.92,1678.0,65.0,0.062681,448.0,2270.75,...,93.795,65.0,1.450615,1.443000,0.902140,175.0,False,3,1,0.9
2,gjokur_ja,3,867.60,69.0,59864.40,69.0,580.0,1.000000,69.0,69.00,...,832.500,580.0,1.495862,1.435345,0.833730,138.0,False,4,0,1.0
3,hammer,1,66.54,3339.0,222177.06,3339.0,45.0,1.000000,3339.0,2672.25,...,62.890,45.0,1.478667,1.397556,1.056624,4794.0,False,4,0,1.0
4,hot_dogs,0,26.25,2240.0,58800.00,2240.0,18.0,0.977123,2240.0,1974.00,...,25.365,18.0,1.458333,1.409167,0.958358,3306.0,False,4,0,1.0
5,ice_cream,1,10.68,2864.0,30587.52,2864.0,8.0,0.983213,2864.0,2265.75,...,11.095,8.0,1.335000,1.386875,1.004914,3740.0,False,4,0,1.0
6,knives,2,52.51,1356.0,71203.56,3236.0,38.0,0.249838,1356.0,2930.75,...,55.205,38.0,1.381842,1.452763,0.949712,1838.0,False,4,0,1.0
7,laderhosen,1,316.95,345.0,109347.75,1686.0,220.0,0.064665,345.0,2524.25,...,316.965,220.0,1.440682,1.440750,0.844659,331.0,False,4,0,1.0


In [23]:
# === Price rule (anchored + recency/dead guards) =============================
def price_next(r):
    p0 = float(r["p_anchor"] or 0.0)
    sp = float(r["supplier_price_ref"] or 0.0)
    st = float(r["ST_EMA"] or 0.0)
    dead = bool(r["is_dead"])
    rec = float(r["recency_multiplier"] or 1.0)

    # anchored default
    p = p0
    if dead:
        p = p0 * (1.0 - PRICE_DOWN_ZERO)
    elif st >= 0.85:
        p = p0 * (1.0 + PRICE_UP_SOLDOUT)
    elif 0.65 <= st < 0.85:
        p = p0 * (1.0 + PRICE_UP_GOOD)
    elif st <= 0.25 and (r["amount_avg"] or 0) > 0:
        p = p0 * (1.0 - PRICE_DOWN_BAD)

    # Recency decay (penalize stale items)
    p *= rec

    # Margin floor
    min_allowed = sp*(1.0 + MIN_MARGIN_RATE) if sp>0 else 0.0
    p = max(p, min_allowed)

    # Per-round caps (tighter for living items, looser for dead)
    cap_up   = p0*(1.0 + (ROUND_CAP_DEAD if dead else ROUND_CAP_STD))
    cap_down = p0*(1.0 - (ROUND_CAP_DEAD if dead else ROUND_CAP_STD))
    p = min(max(p, cap_down), cap_up)

    return round(float(p), 2)

In [24]:
# === Amount rule (trend + risk cap + recency/dead) ===========================
def amount_next(r):
    st_ema = float(r["ST_EMA"] or 0.0)
    qty_max = float(r["qty_max"] or 0.0)
    amt_avg = float(r["amount_avg"] or 0.0)
    risk    = float(r["RISK"] or 1.0)
    dead    = bool(r["is_dead"])
    rec     = float(r["recency_multiplier"] or 1.0)

    # Base: over observed max demand
    base = math.ceil(qty_max * AMT_BASE_BUFFER)
    if st_ema > 0.9:
        base = math.ceil(base * AMT_BOOST_HOT)
    if dead:
        base = max(0, round(amt_avg * AMT_DEAD_FACTOR))

    # Risk cap: higher risk => smaller cap multiplier
    cap_mult = np.clip(1.0 + 0.5/(risk if risk>0 else 1.0), RISK_CAP_MIN, RISK_CAP_MAX)
    cap = amt_avg * cap_mult
    if cap > 0:
        base = min(base, math.ceil(cap))

    # Recency downscale
    base = int(base * rec)

    return int(max(0, base))

In [25]:
# === Schedule selection by ROI ==============================================
# Use md: version-level revenue and staff cost
rev_by_v   = md.groupby("version")["revenue"].sum()
staff_by_v = md.groupby("version")["version_staff_cost"].max()  # one per version
roi = (rev_by_v / staff_by_v.replace(0, np.nan)).dropna()

# Consider only historical weeks
roi_hist = roi[roi.index.isin(HIST_VERS)]
if not roi_hist.empty:
    sched_src_v = int(roi_hist.idxmax())
else:
    # fallback to latest historical with a schedule file
    candidates = [v for v in reversed(HIST_VERS) if (SCHEDULES_DIR / f"schedules_{v}.json").exists()]
    sched_src_v = candidates[0] if candidates else HIST_VERS[-1]

with open(SCHEDULES_DIR / f"schedules_{sched_src_v}.json", "r", encoding="utf-8") as f:
    schedules_next = json.load(f)

print(f"Schedule source version (best ROI among history): v{sched_src_v}")

Schedule source version (best ROI among history): v0


In [26]:
# === Generate JSONs & Audit ==================================================
prices_next  = {r["product"]: price_next(r)  for _, r in ref.iterrows()}
amounts_next = {r["product"]: amount_next(r) for _, r in ref.iterrows()}

# Write decisions
(DATA_DIR/"prices").mkdir(parents=True, exist_ok=True)
(DATA_DIR/"amounts").mkdir(parents=True, exist_ok=True)
(DATA_DIR/"schedules").mkdir(parents=True, exist_ok=True)

with open(DATA_DIR/"prices"/f"prices_{V_NEXT}.json", "w", encoding="utf-8") as f:
    json.dump(prices_next, f, indent=2, ensure_ascii=False)
with open(DATA_DIR/"amounts"/f"amounts_{V_NEXT}.json", "w", encoding="utf-8") as f:
    json.dump(amounts_next, f, indent=2, ensure_ascii=False)
with open(DATA_DIR/"schedules"/f"schedules_{V_NEXT}.json", "w", encoding="utf-8") as f:
    json.dump(schedules_next, f, indent=2, ensure_ascii=False)

# Audit table
audit = ref[["product","anchor_v","p_anchor","qty_max","amount_avg","ST_EMA","RISK","is_dead","weeks_since_sale","recency_multiplier"]].copy()
audit["price_next"]  = audit["product"].map(prices_next)
audit["amount_next"] = audit["product"].map(amounts_next)
audit["d_price_vs_anchor_%"] = (audit["price_next"] - audit["p_anchor"]) / audit["p_anchor"] * 100

audit_path = OUT_DIR / f"decision_v{V_NEXT}_audit.csv"
audit.to_csv(audit_path, index=False)

print("Generated decisions:")
print(" -", DATA_DIR/"prices"/f"prices_{V_NEXT}.json")
print(" -", DATA_DIR/"amounts"/f"amounts_{V_NEXT}.json")
print(" -", DATA_DIR/"schedules"/f"schedules_{V_NEXT}.json")
print("Audit:", audit_path)

display(audit.sort_values(["is_dead","weeks_since_sale","RISK"], ascending=[False, False, False]).head(15))

Generated decisions:
 - data/prices/prices_5.json
 - data/amounts/amounts_5.json
 - data/schedules/schedules_5.json
Audit: output/decision_v5_audit.csv


,product,anchor_v,p_anchor,qty_max,amount_avg,ST_EMA,RISK,is_dead,weeks_since_sale,recency_multiplier,price_next,amount_next,d_price_vs_anchor_%
1,dinosaur,1,94.29,448.0,2270.75,0.062681,0.902140,False,1,0.9,89.58,484,-4.995227
8,mattress,0,372.34,2161.0,2699.00,0.328243,1.145047,False,0,1.0,372.34,2594,0.000000
9,monster,3,20.94,2592.0,2266.00,0.934545,1.090559,False,0,1.0,21.57,3305,3.008596
10,nails,1,8.39,3381.0,2828.50,1.000000,1.068705,False,0,1.0,8.64,4152,2.979738
11,rice_porridge,2,20.46,3259.0,2548.25,1.000000,1.064711,False,0,1.0,21.07,3745,2.981427
3,hammer,1,66.54,3339.0,2672.25,1.000000,1.056624,False,0,1.0,68.54,3937,3.005711
12,sunscreen,2,52.28,2434.0,2029.50,1.000000,1.047068,False,0,1.0,53.85,2999,3.003060
5,ice_cream,1,10.68,2864.0,2265.75,0.983213,1.004914,False,0,1.0,11.00,3394,2.996255
0,batteries,2,15.85,2737.0,2131.25,1.000000,0.982917,False,0,1.0,16.33,3197,3.028391
4,hot_dogs,0,26.25,2240.0,1974.00,0.977123,0.958358,False,0,1.0,27.04,2957,3.009524


In [27]:
# === Quick anomaly checks ====================================================
def show_anomalies(filters=ANOMALY_FILTERS, df=audit):
    flt = None
    for token in filters:
        m = df["product"].str.contains(token, case=False, na=False)
        flt = m if flt is None else (flt | m)
    res = df[flt] if flt is not None else pd.DataFrame()
    if res.empty:
        print("No products matched filters:", filters)
    else:
        print("Anomaly candidates:")
        display(res.sort_values(["is_dead","weeks_since_sale","RISK"], ascending=[False, False, False]))

show_anomalies()

Anomaly candidates:


,product,anchor_v,p_anchor,qty_max,amount_avg,ST_EMA,RISK,is_dead,weeks_since_sale,recency_multiplier,price_next,amount_next,d_price_vs_anchor_%
1,dinosaur,1,94.29,448.0,2270.75,0.062681,0.902140,False,1,0.9,89.58,484,-4.995227
8,mattress,0,372.34,2161.0,2699.00,0.328243,1.145047,False,0,1.0,372.34,2594,0.000000


### Notes & Rationale

- **Anchor pricing** prevents using a stale or depressed last-week price. We anchor at the best-selling week.
- **Recency decay** reduces both price and amount for products that haven’t sold in a while (e.g., “dinosaur”).
- **Dead-item suppression**: If no sales in the last two weeks, apply stronger reductions and keep amount small.
- **Risk cap**: avoids big stock increases on expensive or volatile items.
- **Schedule ROI**: choose the historical week with the best revenue/staff-cost ratio as the template.
- **Per-round caps** keep changes conservative in a competitive environment.

If any product still looks odd in the audit, tweak:
- `RECENCY_DECAY` (0.05–0.15), `RECENCY_MIN` (0.5–0.8),
- amount caps (`RISK_CAP_MIN/MAX`),
- price caps (`ROUND_CAP_STD/DEAD`).